### Importing Depencies

In [1]:
import nltk
import numpy as np
import random
import string

nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

### Reading the Data

In [2]:
f = open('chatbot.txt', 'r', errors='ignore')

raw = f.read()
raw = raw.lower()    # converts to lowercase

nltk.download('punkt')    # first-time use only
nltk.download('wordnet')    # first-time use only

sent_tokens = nltk.sent_tokenize(raw)   # converts to list of sentences
word_tokens = nltk.word_tokenize(raw)   # converts to list of words

[nltk_data] Downloading package punkt to /home/elba/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/elba/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Reviewing Tokenization

In [3]:
sent_tokens[:2]

['background\n\nin 1950, alan turing\'s famous article "computing machinery and intelligence" was published,[8] which proposed what is now called the turing test as a criterion of intelligence.',
 'this criterion depends on the ability of a computer program to impersonate a human in a real-time written conversation with a human judge to the extent that the judge is unable to distinguish reliably—on the basis of the conversational content alone—between the program and a real human.']

In [4]:
word_tokens[:2]

['background', 'in']

### Pre-Processing Raw Text

In [5]:
lemmer = nltk.stem.WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

### Keyword Matching

In [6]:
GREETING_INPUTS = ('hello', 'hi', 'greetings', 'sup', "what's up", 'hey', )
GREETING_RESPONSES = ['hi', 'hey', '*nods*', 'hi there', 'hello', 'I am glad! You are talking to me']

def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

### Generating Response

In [7]:
# Converts a collection of raw documents to a matrix of TF-IDF features
from sklearn.feature_extraction.text import TfidfVectorizer

# Finds similarities between words entered by user and words in corpus
from sklearn.metrics.pairwise import cosine_similarity

# Define a function to search user's utterance for known keywords; returns one of possible responses
def response(user_response):
    robo_response = ''
    sent_tokens.append(user_response)
    
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    
    if (req_tfidf == 0):
        robo_response = robo_response + "I am sorry! I don't understand you."
        return robo_response
    else:
        robo_response = robo_response + sent_tokens[idx]
        return robo_response

In [8]:
# Feed the Bot lines to say while starting and ending a conversation (depending on user's input)
flag = True
print('ROBO: My name is Robo. I will answer your questions about Chatbots. If you want to exit, type Bye!')
while (flag == True):
    user_response = input()
    user_response = user_response.lower()
    if (user_response != 'bye'):
        if (user_response == 'thanks' or user_response == 'thank you'):
            flag = False
            print('ROBO: You are welcome.')
        else:
            if (greeting(user_response) != None):
                print('ROBO: ' + greeting(user_response))
            else:
                print('ROBO: ', end='')
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag = False
        print('ROBO: Bye! Have a good day.')

ROBO: My name is Robo. I will answer your questions about Chatbots. If you want to exit, type Bye!
Hello
ROBO: hey
Thanks
ROBO: You are welcome.
